# Working with Time Zones

## Python's Time Zone Model

The `tzinfo` class is an abstract base class, and you are required to implement three methods:

```python
class tzinfo:
    def tzname(self, dt):
        raise NotImplementedError()

    def utcoffset(self, dt):
        raise NotImplementedError()
        
    def dst(self, dt):
        raise NotImplementedError()
        
    ...
```

- `tzname`: Return the name or abbreviation for the time zone at the given datetime.
- `utcoffset`: Return a `datetime.timedelta` representing the offset from UTC at the given datetime.
- `dst`: Return a `datetime.timedelta` representing the amount of the daylight saving time offset at a given time. *This method is rarely useful*.

The reason these are *methods* rather than fixed values is that a `tzinfo` represents a mapping between a datetime and the set of rules that applies at that time in this zone. The same concrete `tzinfo` class is used for many `datetime`s.

### Example: Eastern Time

In [1]:
from dateutil import tz
from datetime import datetime, timedelta

In [2]:
def print_tzinfo(dt):
    """Convenience function for printing the time zone information of a datetime"""
    print("dt.tzname: " + str(dt.tzname()))
    print("dt.utcoffset: " + str(dt.utcoffset() / timedelta(hours=1)) + " hours")
    print("dt.dst: " + str(dt.dst() / timedelta(hours=1)) + " hours")

In [3]:
EASTERN = tz.gettz("America/New_York")

In [4]:
dt_std = datetime(2019, 3, 1, 12, tzinfo=EASTERN)

In [5]:
print_tzinfo(dt_std)

dt.tzname: EST
dt.utcoffset: -5.0 hours
dt.dst: 0.0 hours


In [6]:
dt_dst = datetime(2019, 4, 1, 12, tzinfo=EASTERN)

In [7]:
print_tzinfo(dt_dst)

dt.tzname: EDT
dt.utcoffset: -4.0 hours
dt.dst: 1.0 hours


### Exercise: Implement a UTC time zone class

The simplest `tzinfo` classes to implement are fixed offsets from UTC and UTC itself, which have offsets that do not change as a function of `datetime`. To practice a basic tzinfo implementation, implement a class `UTC` representing the UTC time zone.

In [8]:
from datetime import tzinfo

In [9]:
from datetime import timezone

In [10]:
datetime(2019, 1, 1, tzinfo=timezone.utc).dst()

In [11]:
class UTC(tzinfo):
    def tzname(self, dt):
        return "UTC"
    
    def utcoffset(self, dt):
        return timedelta(0)
    
    def dst(self, dt):
        return None

In [12]:
# Tests
test_cases = [
    datetime(1, 1, 1),
    datetime(1857, 2, 7),
    datetime(1970, 1, 1),
    datetime(2010, 3, 21, 2, 16)
]

utc = UTC()
for dt in test_cases:
    dt_act = dt.replace(tzinfo=utc)
    dt_exp = dt.replace(tzinfo=timezone.utc)
    
    assert dt_act.tzname() == dt_exp.tzname(), f'tznames must match for {dt}'
    assert dt_act.utcoffset() == dt_exp.utcoffset(), f'utcoffset must match for {dt}'
    assert dt_act.dst() == dt_exp.dst(), f'dst must match for {dt}'

## Creating and working with time-zone objects

The standard way to create a datetime literal is to attach it to the constructor by passing it to the `tzinfo` argument of the constructor:

In [13]:
from helper_functions import print_dt_tzinfo

SyntaxError: invalid syntax (helper_functions.py, line 5)

In [ ]:
dt = datetime(2017, 8, 11, 14, tzinfo=tz.gettz('America/New_York'))
print_dt_tzinfo(dt)

If you have a naive wall time or a wall time in another zone that you want to translate without shifting, use `datetime.replace`:

In [ ]:
print_dt_tzinfo(dt.replace(tzinfo=tz.gettz('America/Los_Angeles')))

If you have an *absolute* time, in UTC or otherwise, and you want to represent it in another timezone, use `datetime.astimezone`:

In [ ]:
print_dt_tzinfo(dt.astimezone(tz.gettz('America/Los_Angeles')))